In [110]:
from numpy.core import (
    array, asarray, zeros, empty, empty_like, intc, single, double,
    csingle, cdouble, inexact, complexfloating, newaxis, all, Inf, dot,
    add, multiply, sqrt, fastCopyAndTranspose, sum, isfinite,
    finfo, errstate, geterrobj, moveaxis, amin, amax, product, abs,
    atleast_2d, intp, asanyarray, object_, matmul,
    swapaxes, divide, count_nonzero, isnan
)
from numpy.core.multiarray import normalize_axis_index
from numpy.lib.twodim_base import triu, eye
from numpy.linalg import lapack_lite, _umath_linalg, eigh, svd, LinAlgError

import numpy as np
import scipy as sp
import scipy.linalg

In [23]:
def _isEmpty2d(arr):
    # check size first for efficiency
    return arr.size == 0 and product(arr.shape[-2:]) == 0

def _makearray(a):
    new = asarray(a)
    wrap = getattr(a, "__array_prepare__", new.__array_wrap__)
    return new, wrap

def transpose(a):
    """
    Transpose each matrix in a stack of matrices.
    Unlike np.transpose, this only swaps the last two axes, rather than all of
    them
    Parameters
    ----------
    a : (...,M,N) array_like
    Returns
    -------
    aT : (...,N,M) ndarray
    """
    return swapaxes(a, -1, -2)

In [85]:
uhh = np.empty((0, 1, 1))
print(uhh)
_isEmpty2d(uhh)

[]


False

In [111]:
def pinv(a, rcond=1e-15 ):
    """
    Compute the (Moore-Penrose) pseudo-inverse of a matrix.
    Calculate the generalized inverse of a matrix using its
    singular-value decomposition (SVD) and including all
    *large* singular values.
    .. versionchanged:: 1.14
       Can now operate on stacks of matrices
    Parameters
    ----------
    a : (..., M, N) array_like
        Matrix or stack of matrices to be pseudo-inverted.
    rcond : (...) array_like of float
        Cutoff for small singular values.
        Singular values smaller (in modulus) than
        `rcond` * largest_singular_value (again, in modulus)
        are set to zero. Broadcasts against the stack of matrices
    Returns
    -------
    B : (..., N, M) ndarray
        The pseudo-inverse of `a`. If `a` is a `matrix` instance, then so
        is `B`.
    Raises
    ------
    LinAlgError
        If the SVD computation does not converge.
    Notes
    -----
    The pseudo-inverse of a matrix A, denoted :math:`A^+`, is
    defined as: "the matrix that 'solves' [the least-squares problem]
    :math:`Ax = b`," i.e., if :math:`\\bar{x}` is said solution, then
    :math:`A^+` is that matrix such that :math:`\\bar{x} = A^+b`.
    It can be shown that if :math:`Q_1 \\Sigma Q_2^T = A` is the singular
    value decomposition of A, then
    :math:`A^+ = Q_2 \\Sigma^+ Q_1^T`, where :math:`Q_{1,2}` are
    orthogonal matrices, :math:`\\Sigma` is a diagonal matrix consisting
    of A's so-called singular values, (followed, typically, by
    zeros), and then :math:`\\Sigma^+` is simply the diagonal matrix
    consisting of the reciprocals of A's singular values
    (again, followed by zeros). [1]_
    References
    ----------
    .. [1] G. Strang, *Linear Algebra and Its Applications*, 2nd Ed., Orlando,
           FL, Academic Press, Inc., 1980, pp. 139-142.
    Examples
    --------
    The following example checks that ``a * a+ * a == a`` and
    ``a+ * a * a+ == a+``:
    >>> a = np.random.randn(9, 6)
    >>> B = np.linalg.pinv(a)
    >>> np.allclose(a, np.dot(a, np.dot(B, a)))
    True
    >>> np.allclose(B, np.dot(B, np.dot(a, B)))
    True
    """
    a, wrap = _makearray(a)
    rcond = asarray(rcond)
    if _isEmpty2d(a):
        m, n = a.shape[-2:]
        res = empty(a.shape[:-2] + (n, m), dtype=a.dtype)
        return wrap(res)
    a = a.conjugate()
    u, s, vt = svd(a, full_matrices=False)

    # discard small singular values
    cutoff = rcond[..., newaxis] * amax(s, axis=-1, keepdims=True)
    large = s > cutoff
    s = divide(1, s, where=large, out=s)
    s[~large] = 0

    res = matmul(transpose(vt), multiply(s[..., newaxis], transpose(u)))
    return wrap(res)


def pinvh(a, rcond=1e-15):
    """
    Compute the (Moore-Penrose) pseudo-inverse of a Hermitian matrix.
    Calculate the generalized inverse of a matrix using its
    eigenvalue decomposition and including all
    *large* singular values.

    Parameters
    ----------
    a : (..., N, N) array_like
        Matrix or stack of Hermitian matrices to be pseudo-inverted.
    rcond : (...) array_like of float
        Cutoff for small singular values.
        Singular values smaller (in modulus) than
        `rcond` * largest_singular_value (again, in modulus)
        are set to zero. Broadcasts against the stack of matrices.
    Returns
    -------
    B : (..., N, N) ndarray
        The pseudo-inverse of `a`. If `a` is a `matrix` instance, then so
        is `B`.
    Raises
    ------
    LinAlgError
        If the eigenvalue computation does not converge.
    Notes
    -----
    The pseudo-inverse of a matrix A, denoted :math:`A^+`, is
    defined as: "the matrix that 'solves' [the least-squares problem]
    :math:`Ax = b`," i.e., if :math:`\\bar{x}` is said solution, then
    :math:`A^+` is that matrix such that :math:`\\bar{x} = A^+b`.
    It can be shown that if :math:`Q_1 \\Sigma Q_2^T = A` is the singular
    value decomposition of A, then
    :math:`A^+ = Q_2 \\Sigma^+ Q_1^T`, where :math:`Q_{1,2}` are
    orthogonal matrices, :math:`\\Sigma` is a diagonal matrix consisting
    of A's so-called singular values, (followed, typically, by
    zeros), and then :math:`\\Sigma^+` is simply the diagonal matrix
    consisting of the reciprocals of A's singular values
    (again, followed by zeros). [1]_
    References
    ----------
    .. [1] G. Strang, *Linear Algebra and Its Applications*, 2nd Ed., Orlando,
           FL, Academic Press, Inc., 1980, pp. 139-142.
    Examples
    --------
    The following example checks that ``a * a+ * a == a`` and
    ``a+ * a * a+ == a+``:
    >>> a = np.random.randn(6, 6)
    >>> a = a @ a.T  # To symmetrize it
    >>> B = np.linalg.pinvh(a)
    >>> np.allclose(a, np.matmul(a, np.matmul(B, a)))
    True
    >>> np.allclose(B, np.matmul(B, np.matmul(a, B)))
    True
    """
    a, wrap = _makearray(a)
    rcond = asarray(rcond)
    if _isEmpty2d(a):
        m, n = a.shape[-2:]
        if m != n:
            raise LinAlgError('Last 2 dimensions of the array must be square')
        res = empty(a.shape[:-2] + (n, m), dtype=a.dtype)
        return wrap(res)
    s, u = eigh(a, UPLO='L')

    # discard small singular values
    cutoff = rcond[..., newaxis] * amax(s, axis=-1, keepdims=True)
    large = s > cutoff
    s = divide(1, s, where=large, out=s)
    s[~large] = 0

    res = matmul(u, multiply(s[..., newaxis], transpose(np.conjugate(u))))
    return wrap(res)


In [115]:
# a = np.random.randn(60, 60) + 1j * np.random.randn(60, 60)
# a = a @ transpose(np.conjugate(a))
a = np.empty((5, 0, 0))
print(a)
# print(np.allclose(transpose(np.conjugate(a)), a))

[]


In [116]:
# a = np.random.randn(9, 6)
B = np.linalg.pinv(a)
print(B)
print(np.allclose(a, a @ B @ a))

[]
True


In [118]:
# a = a @ transpose(a)
B = pinvh(a)
print(B, B.shape)
print(np.allclose(a, a @ B @ a))
print(np.allclose(B, np.matmul(B, np.matmul(a, B))))

[] (5, 0, 0)
True
True


In [53]:
print(np.allclose(pinvh(a), np.linalg.pinv(a)))
# print(np.allclose(np.array([sp.linalg.pinvh(aa) for aa in a]), np.linalg.pinv(a)))
print(np.allclose(sp.linalg.pinvh(a), np.linalg.pinv(a)))

%timeit np.linalg.pinv(a)
%timeit pinvh(a)
# %timeit np.array([sp.linalg.pinvh(aa) for aa in a])
%timeit sp.linalg.pinvh(a, check_finite=False)

True
True
725 µs ± 4.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
404 µs ± 7.08 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
514 µs ± 13.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
np.allclose(transpose(np.conjugate(a)), a)





True

In [ ]:
class PinvhCases(HermitianTestCase,
                 HermitianGeneralizedTestCase):

    def do(self, a, b, tags):
        a_ginv = linalg.pinvh(a)
        # `a @ a_ginv == I` does not hold if a is singular
        dot = matmul
        assert_almost_equal(dot(dot(a, a_ginv), a), a, single_decimal=5, double_decimal=11)
        assert_(consistent_subclass(a_ginv, a))


class TestPinvh(PinvhCases):
    pass